# Bitcoin Price Prediction - UI

## Installing Dependencies

In [ ]:
! pip install gradio -q

## Code

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import model_from_json
import gradio as gr
from keras.models import load_model
import datetime

def predict_prices(start_date, n_days):
    # Basic input validation
    if not start_date:
        return ["Please provide a starting date."]
    if not str(n_days).isdigit():
        return ["Please enter a valid integer for the number of days."]
    if int(n_days) < 1:
        return ["The number of days must be at least 1."]
    if int(n_days) > 31:
        return ["The number of days cannot exceed 31. Please enter a number of days that is 31 or less."]

    try:
        start_date = pd.to_datetime(start_date, dayfirst=True)
    except ValueError:
        return ["Invalid date format. Please use DD-MM-YYYY."]

    current_date = pd.to_datetime("today").normalize()  # Ensure time part is stripped off for comparison
    if start_date < current_date or start_date > current_date + pd.DateOffset(days=30):  # Adjusted to include today in the 31 days
        return ["The starting date must be today or within the next 30 days."]

    # Load model and data
    with open("/content/model.json", "r") as json_file:
        loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)
    model.load_weights("/content/model.h5")
    df = pd.read_csv('/content/BTC-USD.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.dropna(inplace=True)
    df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')

    # Ensure the start_date is within or can be extended from the dataset
    latest_date = df['Date'].max()
    if start_date > latest_date:
        # Predict from the latest available data
        start_date = latest_date

    look_back = 15
    start_index = df[df['Date'] == start_date].index.tolist()[0] if start_date in df['Date'].values else -1
    if start_index == -1:
        return ["Starting date is before the beginning of the dataset."]

    data = df['Close'].values[start_index-look_back:start_index].reshape(-1, 1)
    min_max_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = min_max_scaler.fit_transform(data)
    x_input = scaled_data.reshape(1, -1)
    temp_input = list(x_input[0])
    predicted_prices = []

    i = 0
    while i < n_days:
        if len(temp_input) > look_back:
            x_input = np.array(temp_input[-look_back:])
        x_input = x_input.reshape((1, look_back, 1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.append(yhat[0][0])
        predicted_price = min_max_scaler.inverse_transform(yhat)[0][0]
        predicted_prices.append(f"Predicted price for Day {i+1} is {predicted_price:.2f}")
        i += 1

    return predicted_prices

# Gradio interface setup
date_input = gr.Textbox(label="Enter Starting date", placeholder="DD-MM-YYYY")
days_input = gr.Number(label="Enter the number of days")
output = gr.Textbox(label="Predicted prices")
interface = gr.Interface(
    fn=predict_prices,
    inputs=[date_input, days_input],
    outputs=output,
    title="Bitcoin Price Prediction",
    description="Enter the starting date and the number of days to predict Bitcoin prices, up to a maximum of 31 days."
)

# Launch the Gradio interface
interface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://866dd1a1a77f6dc3ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
